In [ ]:
from pathlib import Path
from math import ceil
from textwrap import wrap
from io import BytesIO
from typing import Union, IO, Iterator, Callable, Tuple, List, Any, Optional, Iterable
import re
import gzip
import logging
from collections import OrderedDict
from nptyping import NDArray
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.units import Quantity
import hpmoc as hpm
from hpmoc.healpy import healpy as hp
from hpmoc import PartialUniqSkymap
from hpmoc import utils as ut
from hpmoc.utils import *
from hpmoc.fits import (
    next_header,
    next_hdu,
    bintable_dtype,
    _next_header_blocks,
    read_bintable_chunks,
    extract_probdensity,
    calculate_max_rows_read,
    HEADER_NON_META,
    load_ligo,
)

LOGGER = logging.getLogger(__name__)
DATA = Path(".").absolute().parent/"tests"/"data"

NUNIQ_FITS = DATA/'S200105ae.fits'
NUNIQ_FITS_GZ = DATA/'S200105ae.fits.gz'
BAYESTAR_NEST_FITS_GZ_1024 = DATA/'S200316bj-1-Preliminary.fits.gz'
CWB_NEST_FITS_GZ_128 = DATA/'S200114f-3-Initial.fits.gz'
CWB_RING_FITS_GZ_128 = DATA/'S200129m-3-Initial.fits.gz'

In [ ]:
def compare_new_old_ligo_io_read(infile, mask: PartialUniqSkymap = None):
    old = (mask or PartialUniqSkymap).read(infile, strategy='ligo_old')
    new = (mask or PartialUniqSkymap).read(infile, strategy='ligo')
    assert (old == new).s.all(), f"old ({old}) != new ({new}) for {infile}"
    cov_old = ut.uniq_minimize(old.u)[0]
    cov_new = ut.uniq_minimize(new.u)[0]
    assert (cov_old == cov_new).all(), \
        f"old cov ({cov_old}) != new cov ({cov_new}) for {infile}"
    for (k, v), (ko, vo) in zip(new.meta.items(), old.meta.items()):
        if k != 'HISTORY':
            assert k == ko, f"meta key mismatch: {k} != {ko} for {infile}"
            assert v == vo, f"meta val mismatch: {v} != {vo} for {infile}"

In [ ]:
compare_new_old_ligo_io_read(CWB_RING_FITS_GZ_128)

In [ ]:
moo = PartialUniqSkymap.read(CWB_RING_FITS_GZ_128, strategy='ligo_old')
mo = PartialUniqSkymap.read(CWB_RING_FITS_GZ_128, strategy='ligo')

In [ ]:
plt.suptitle

In [ ]:
ax = mo.plot(fig={'figsize': (6, 5)}, cbar={'pad': 0.05}, cmap='bone_r')
mo.orders(as_skymap=True).plot(projection=ax.wcs, ax=ax,
                               alpha=0.15, cmap='rainbow', cbar={'pad': 0.1})
_ = ax.set_title("S200129m-3-Initial")

In [ ]:
ax.set_title?


In [ ]:
mo.meta

In [ ]:
PartialUniqSkymap.read(NUNIQ_FITS, strategy='ligo').meta

In [ ]:
plt.colorbar?

In [ ]:
for src in [NUNIQ_FITS, NUNIQ_FITS_GZ, BAYESTAR_NEST_FITS_GZ_1024,
            CWB_NEST_FITS_GZ_128, CWB_RING_FITS_GZ_128]:
    compare_new_old_ligo_io_read(src)

In [ ]:
fig = plt.figure(dpi=50)
mcwb = PartialUniqSkymap.read(CWB_NEST_FITS_GZ_128, strategy='ligo')
mcwb.plot(projection='CAR', fig=fig, cbar=True)

In [ ]:
mcwb

In [ ]:
mo = PartialUniqSkymap.read(NUNIQ_FITS, strategy='ligo')

In [ ]:
mo.plot(projection='plate-caree')

In [ ]:
mot

In [ ]:
with gzip.open(CWB_NEST_FITS_GZ_128, 'rb') as f:
    [[u, s, meta]] = load_ligo(f)
    mo = PartialUniqSkymap(s, u, meta=meta, name='PROBDENSITY')

In [ ]:
mo

In [ ]:
ax = mo.o⃗(as_skymap=True).plot(cbar=True)#projection='CAR', cmap='rainbow', cbar={'orientation': 'horizontal', 'aspect': 40})

In [ ]:
mo.plot()

In [ ]:
moo = PartialUniqSkymap.read(CWB_NEST_FITS_GZ_128, strategy='ligo')

In [ ]:
moo.to_table().meta

In [ ]:
from astropy.table import meta

In [ ]:
mo.meta

In [ ]:
mo.plot(projection='CAR', cbar=True)

In [ ]:
str(mo.s⃗.unit)

In [ ]:
hp.nside2npix(hp.order2nside(7))

In [ ]:
mo